In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.6.0'

# Data Pre-Processing

Open **dan_train.csv** file and split the games into a list.
Every row of csv: `DL0000000001,B,B[pd],W[dp],B[pp],W[dc],B[de],...`. 

Columns are:

    1. DL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [3]:
df = open('./Training Dataset/dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
colors = [i.split(',',2)[1] for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [4]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [5]:
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[last_move_row,last_move_column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [6]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 100160, Total Moves: 22853380


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [7]:
def data_generator(games, batch_size):
    def generator():
        x_batch = [] # Initialize data batch
        y_batch = [] # Initialize target batch
        for game_i, game in enumerate(games): # Iterate through games
            moves_list = game.split(',')
            for count, move in enumerate(moves_list):
                if colors[game_i] == move[0]:
#                     print(count)
                    x_batch.append(prepare_input(moves_list[:count]))
                    y_batch.append(prepare_label(moves_list[count]))
                    if len(x_batch) == batch_size: # Yield when reached batch size
                        yield np.array(x_batch), tf.one_hot(np.array(y_batch), depth=19*19)
                        x_batch = []
                        y_batch = []
    return generator

batch_size = 128
generator = data_generator(games, batch_size)
dataset = tf.data.Dataset.from_generator(generator, 
                                         output_types=(tf.float32, tf.float32),
                                         output_shapes=(tf.TensorShape((batch_size,19,19,4)),tf.TensorShape((batch_size,361)))
                                        )
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [24]:
# x.shape

(118777, 19, 19, 4)

In [25]:
# y.shape

(118777,)

In [26]:
# y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [27]:
# x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [8]:
def create_model():
    inputs = Input(shape=(19, 19, 4))
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    outputs = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)
    
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [9]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 19, 19, 4)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 19, 19, 64)        12608     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 64)        200768    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 19, 19, 64)        102464    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 19, 64)        102464    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 19, 19, 64)        36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 19, 19, 64)        36928 

In [10]:
history = model.fit(
    dataset,
    epochs = 1
)

  80894/Unknown - 2499s 31ms/step - loss: 2.8468 - accuracy: 0.3506

KeyboardInterrupt: 

In [11]:
model.save('./model_dan_one_epoch.h5')

## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!